In [27]:
import pandas as pd
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizerBase,
    pipeline,
)
import os
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct")

In [28]:
os.listdir('../../data/training_data')

['D10001.csv',
 'D90000.csv',
 'D20002.csv',
 'training_data.csv',
 'D20003.csv',
 'D20001.csv',
 'D30004.csv',
 'D30001.csv',
 'D70005_LIGHT.csv',
 'D30002.csv',
 'D30003.csv',
 'clem_top_10_models_data_all_successful_episodes_no_preprocessing.csv',
 'D70005.csv',
 'D70004.csv',
 'D70001.csv',
 'D70003.csv',
 'D70002.csv',
 'clem_top_10_models_data_all_successful_episodes_individual_turns_adapted_imagegame.csv',
 '.ipynb_checkpoints',
 'clem_top_10_models_data_all_successful_episodes_individual_turns.csv',
 'D50001.csv',
 'D50003.csv',
 'D8000X.csv',
 'D50002.csv',
 'D50006.csv',
 'D50007.csv',
 'D50005.csv',
 'D90000_LIGHT.csv',
 'D50004.csv']

In [29]:
all_datasets = ['D90000_LIGHT.csv']

In [30]:
for dataset in all_datasets:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: eval(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    df.to_csv(path, index=False)

In [31]:
df.head()

,game,game_id,model,benchmark_version,experiment,episode,Aborted,Lose,Success,chat,target,player,text
0,referencegame,21,gpt-4-0125-preview-t0.0--gpt-4-0125-preview-t0.0,v1.5,2_diagonal_grids,episode_21,0,0,1,"[{'role': 'user', 'content': 'You are given th...",first,player 1,<|begin_of_text|><|start_header_id|>system<|en...
1,referencegame,18,gpt-4-0125-preview-t0.0--gpt-4-0125-preview-t0.0,v1.5,5_random_grids,episode_18,0,0,1,"[{'role': 'user', 'content': 'You are given th...",first,player 1,<|begin_of_text|><|start_header_id|>system<|en...
2,referencegame,23,Meta-Llama-3.1-405B-Instruct-Turbo-t0.0--Meta-...,v1.6,4_shape_grids,episode_23,0,0,1,"[{'role': 'user', 'content': 'You are given th...","['third', '3rd', '3']",player 1,<|begin_of_text|><|start_header_id|>system<|en...
3,referencegame,6,gpt-4-0125-preview-t0.0--gpt-4-0125-preview-t0.0,v1.5,1_line_grids_columns,episode_6,0,0,1,"[{'role': 'user', 'content': 'You are given th...",first,player 1,<|begin_of_text|><|start_header_id|>system<|en...
4,referencegame,6,Meta-Llama-3.1-405B-Instruct-Turbo-t0.0--Meta-...,v1.6,2_diagonal_grids,episode_6,0,0,1,"[{'role': 'user', 'content': 'You are given th...","['first', '1st', '1']",player 2,<|begin_of_text|><|start_header_id|>system<|en...
